### GridSearch appt_v2

#### Logistic Regression

In [ ]:
kf = StratifiedKFold(n_splits=5,random_state=60,shuffle=True)

grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression(solver='liblinear')
logreg_cv=GridSearchCV(logreg,grid,cv=kf,n_jobs=-1,verbose=1)
logreg_cv.fit(Xv2_train_std,yv2_train)

In [ ]:
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

In [ ]:
a = list(zip(Xv2_cols,logreg_cv.best_estimator_.coef_[0]))
for coef in Sort_Tuple(a):
    print(coef)

#### KNN

In [ ]:
grid_params={'n_neighbors':[25,31,35,40],
             'weights':['uniform','distance'],
             'metric':['euclidean','manhattan']}
kneighbors=KNeighborsClassifier()
kneighbors_cv=GridSearchCV(kneighbors,grid_params,cv=kf,n_jobs=-1,verbose=1)
kneighbors_cv.fit(Xv2_train_std,yv2_train)

In [ ]:
print("tuned hpyerparameters :(best parameters) ",kneighbors_cv.best_params_)
print("accuracy :",kneighbors_cv.best_score_)

In [ ]:
yv2_pred_KNN_cv = kneighbors_cv.predict(Xv2_test)

In [ ]:
print_scores(yv2_test,yv2_pred_KNN_cv)